# Measuring speed of technology adoption data 

#### Run (1) Clean HATCH Dataset
#### Run (2) clean tech df

In [1]:
#good 
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import warnings
import os.path
from scipy import stats
from sklearn.metrics import r2_score


from matplotlib.pyplot import cm

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
from scipy.optimize import curve_fit

##### Input data

In [2]:
all_tech_country = pd.read_csv("all_tech_clean_cumulative_Updated.csv", low_memory = False)
all_tech_country = all_tech_country.drop(all_tech_country.columns[0], axis=1)


In [5]:
os.chdir('/Users/jennagreene/Documents/GitHub/country-tech-growth')


In [6]:
# Divide the dataframe into summary variables and non-summary variables
tech_timeseries_nosum = all_tech_country.iloc[:,10:]
#tech_timeseries_nosum

tech_timeseries_sum = all_tech_country.iloc[:,:9]
#tech_timeseries_sum


### Define helper functions

- find_first_year - returns first year in the time series (if not blank)
- find_last_year - returns last year in the time series (if not blank)
- get_time_series - returns the numerical elements of the time series (no description)
- int_columns - converts the columns (years) to integers for calculations
- get_max_delta - returns fastest period of growth on time series

In [7]:
#Find first year function given a dataframe and index (index is the country-technology pairs)

def find_first_year(df, index):
    row_values = df.loc[index] # get row at that index
    for year, value in row_values.items(): #year is the column value and the value is each cell

        if pd.notnull(value) and (value !=0): # Iterate to find first non-zero and non-NAN value in row
            return year
        
    return None #else return None


In [8]:
#Find first year function given a dataframe and index (index is the country-technology pairs)

def find_last_year(df, index):

    row_values = df.loc[index] # get row at that index
    
    for year, value in reversed(list(row_values.items())): #find last value by reversing the list of values
        if pd.notnull(value) and (value!=0):
            return year   
    return None

In [9]:
#Creates years and values of trimmed time series

def get_time_series(df, index):
    first_year = find_first_year(df, index)
    last_year = find_last_year(df, index)
    if first_year is None or last_year is None:
        years = []
        values = []
    else:   
        trimmed_timeseries = df.loc[index, first_year:last_year]
        trimmed_timeseries = trimmed_timeseries.dropna()

        years = trimmed_timeseries.index.tolist()
        years = np.array(years, dtype = int)
        values = trimmed_timeseries.tolist()

    return years, values

In [10]:
def int_columns(x):
    try:
        return int(x)
    except:
        return x

tech_timeseries_nosum.columns = tech_timeseries_nosum.columns.map(int_columns)

#tech_timeseries_nosum


In [11]:
# This helper function finds the year of maximum growth in the time series
# this is a helper function for measuring growth

def find_max_delta(years, values):
    max_delta = float(0)  # Initialize max_delta with zero
    max_delta_year = None  # Initialize max_delta_year with None

    for i in range(1, len(years)):
        delta = values[i] - values[i - 1]  # Calculate the difference in values
        if delta > max_delta:
            max_delta = delta
            max_delta_year = years[i]

    return max_delta_year

### Define Growth Functions

In [12]:
def exponential_func(x, a, b, c):

    return a * np.exp(b * x ) + c

In [13]:
def logistic_func(x, L, k, thalf):
    return L/(1 + np.exp(-k * (x - thalf)))

In [14]:
def gompertz_func(x, L, k, thalf):
    return L * np.exp(- np.exp(-k * (x - thalf)))

In [15]:
def deltaT(k):
     return math.log(81) / k

In [16]:
tech_timeseries_sum

,ID,Spatial Scale,Country Code,Country Name,Technology Name,Metric,Unit,Data Source,Variable
0,Sugar Output_Annual Production_JM,National,JAM,Jamaica,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar
1,Sugar Output_Annual Production_BB,National,BRB,Barbados,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar
2,Sugar Output_Annual Production_CU,National,CUB,Cuba,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar
3,Beer_Annual Production_GB,National,GBR,UK,Beer Production,Annual Production,thousand hectolitres,Mitchell,Annual Production|Beer
4,Sugar Output_Annual Production_MU,National,MUS,Mauritius,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar
...,...,...,...,...,...,...,...,...,...
6027,Crop Harvester_Total Number_UZ,National,UZB,Uzbekistan,Crop Harvester,Total Number,-,CHAT,Number of Units|Crop Harvester
6028,Television_Total Number_BA,National,BIH,Bosnia-Herzegovina,Television,Total Number,-,CHAT,Number of Units|Televisions
6029,BCG Vaccine_Share of Population_AT,National,AUT,Austria,BCG Vaccine,Share of Population,%,UNICEF,Share of Population|Tuberculosis Vaccine
6030,MCV2 Vaccine_Share of Population_DJ,National,DJI,Djibouti,MCV2 Vaccine,Share of Population,%,UNICEF,Share of Population|Measles Vaccine Second Dose


In [17]:
def save_scatter_plot(tech_years, tech_values, tech_name, tech_country):
    plt.figure()
    plt.scatter(tech_years, tech_values)
    plt.xlabel('Years')
    plt.ylabel('Values')
    plt.title(f'Time Series Plot for {tech_name} in {tech_country}')
    plt.savefig(f'error_fitting_figures/{tech_name}_{tech_country}_scatter.png')
    plt.close()

In [18]:
def save_time_series_plot(tech_years, tech_values, exp_fit, log_fit, gomp_fit, tech_name, tech_country, exp_r2, log_R_sq , gomp_r2):
    plt.figure()
    plt.scatter(tech_years, tech_values, label='Time Series')

    # Plot exponential fit if available
    if exp_fit is not None: #and exp_r2 > 0:
            plt.plot(tech_years, exp_fit, label=f'Exponential Fit (R2 = {exp_r2})')
        
    # Plot logistic fit if available
    if log_R_sq is not None:
        plt.plot(tech_years, log_fit, label=f'Logistic Fit (R2 = {log_R_sq})')

    # Plot Gompertz fit if available
    if gomp_fit is not None:
        plt.plot(tech_years, gomp_fit, label=f'Gompertz Fit (R2 = {gomp_r2})')

    plt.xlabel('Years')
    plt.ylabel('Values')
    plt.title(f'Time Series and Fits for {tech_name} in {tech_country}')
    plt.legend()
    plt.savefig(f'all_figures/{tech_name}_{tech_country}_fits.png')
    plt.close()

In [19]:
tech_timeseries_sum


,ID,Spatial Scale,Country Code,Country Name,Technology Name,Metric,Unit,Data Source,Variable
0,Sugar Output_Annual Production_JM,National,JAM,Jamaica,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar
1,Sugar Output_Annual Production_BB,National,BRB,Barbados,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar
2,Sugar Output_Annual Production_CU,National,CUB,Cuba,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar
3,Beer_Annual Production_GB,National,GBR,UK,Beer Production,Annual Production,thousand hectolitres,Mitchell,Annual Production|Beer
4,Sugar Output_Annual Production_MU,National,MUS,Mauritius,Sugar Output,Annual Production,thousand metric tons,Mitchell,Annual Production|Cane Sugar
...,...,...,...,...,...,...,...,...,...
6027,Crop Harvester_Total Number_UZ,National,UZB,Uzbekistan,Crop Harvester,Total Number,-,CHAT,Number of Units|Crop Harvester
6028,Television_Total Number_BA,National,BIH,Bosnia-Herzegovina,Television,Total Number,-,CHAT,Number of Units|Televisions
6029,BCG Vaccine_Share of Population_AT,National,AUT,Austria,BCG Vaccine,Share of Population,%,UNICEF,Share of Population|Tuberculosis Vaccine
6030,MCV2 Vaccine_Share of Population_DJ,National,DJI,Djibouti,MCV2 Vaccine,Share of Population,%,UNICEF,Share of Population|Measles Vaccine Second Dose


In [ ]:
curve_fit(method =  )

### Measure speed of time series

In [20]:
#Create new dataframe of fits
technology_exp_fits = pd.DataFrame(columns=['ID', 'Technology Name',
                                            'Country Code',
                                            'Country Name', 
                                            'Metric',
                                            'Exponential Fit', 
                                            'exponential goodness',
                                            'Logistic Fit',
                                            'Logistic Saturation',
                                            'logistic goodness',
                                            'Gompertz Fit',
                                            'gompertz goodness',
                                            'Delta T Wilson Fit',
                                            'Vintage',
                                            'Number of Points'])

tech_fit_list = [] #initialize empty list of fits

#Create counts of reasons to drop
count_no_points = 0 
count_negative_log = 0 #log
count_above_1 = 0 #log
count_no_log_fit = 0 #log
count_no_gomp_fit = 0 #gomp
count_no_exp_fit = 0 #exp

count_badr2 = 0 #log
count_badr2_exp = 0 # exp # coded
count_badr2_gomp = 0 # gomp
drop_df = pd.DataFrame(columns=['Technology', 'Country', 'Reason for Drop', 'Number of Years'])


# this loop will loop through each time series and measure growth.
for idx, row in tech_timeseries_sum.iterrows():
    
    # summary characteristics for each time series.
    tech_name = tech_timeseries_sum['Technology Name'][idx]
    
    id = tech_timeseries_sum['ID'][idx]
    tech_country = tech_timeseries_sum['Country Name'][idx]
    tech_metric = tech_timeseries_sum['Metric'][idx]
    tech_countrycode = tech_timeseries_sum['Country Code'][idx]
    tech_years, tech_values = get_time_series(tech_timeseries_nosum, idx)

    ## Only fit a function for country-tech pairs that are at least ten points
    if len(tech_years) >= 10:
        
        

        ##### EXPONENTIAL FUNCTION #####

        # Exponential fit initial parameters
        #initial_a =  min(tech_values) #vertical shift start at earliest value
        initial_a = max(tech_values) - min(tech_values)
        initial_b = np.log(tech_values[-1] / tech_values[0]) / (tech_years[-1] - tech_years[0]) #steepness start at CAGR
        #initial_b = 0.04
        initial_c = (tech_values[0]) #horizontal shift start at lowest value
        #initial
        p0 = [initial_a, initial_b, initial_c]


        # Try exponential function
        try:
            exp_params, _ = curve_fit(exponential_func, tech_years, tech_values, p0, maxfev=5000)
            a_fit, b_fit, c_fit = exp_params
            tech_steepness = b_fit
           # print("Name:", tech_name, "ExpFit:", tech_steepness)
            exp_fit = exponential_func(tech_years, *exp_params)
            exp_r2 = r2_score(tech_values, exp_fit) 
            if exp_r2 < 0.95: 
                count_badr2_exp += 1 # add counter for bad R2, will drop.
        
        except Exception as e:
            print(f"Exception occured: {e}")
            tech_steepness = 'err'
            exp_fit = None
            exp_r2 = None
            count_no_exp_fit += 1

        
        ##### LOGISTIC FUNCTION #####
   
        max_change_yr = find_max_delta(tech_years, tech_values)    
            
        # for logistic function fit
        # from Cherp et al paper
        
        #initial parameter for asymptote is highest value in dataset
        initial_L = tech_values[-1]
        
        # initial parameter for inflection year is the yer iwth the biggest change
        initial_thalf = max_change_yr    
        
        # initial k steepness is 0.5
        initial_k = 0.5

        param_start = [initial_L, initial_k, initial_thalf]
        
        # Try logistic function.
        try:
            log_params, _ = curve_fit(logistic_func, tech_years, tech_values, param_start, maxfev=5000)
            l_fit, k_fit, thalf_fit = log_params
            log_steepness = k_fit
            log_saturation = l_fit
            log_fit = logistic_func(tech_years, *log_params)
            
            if log_steepness < 0: # if steepness is negative, will drop.
                count_negative_log += 1

            if log_steepness > 1:
                count_above_1 +=1 
                

            tech_fit = logistic_func(tech_years, l_fit, k_fit, thalf_fit)
            log_R_sq = r2_score(tech_values, tech_fit)
            if log_R_sq < 0.95: 
                count_badr2 += 1 # add counter for bad R2, will drop.

            # Calculate deltaT for each series.
            #deltaT_steepness = (1/k_fit)* np.log(81)
            ### Delta T
            deltaTspeed = deltaT(k_fit)

        except Exception as e:
            log_steepness = 'err'
            deltaTspeed = 'err'
            thalf_fit = 'err'
            log_R_sq = None
            log_fit = None

            # if logistic function can't fit, add here.
            count_no_log_fit += 1
            new_data = pd.DataFrame({'Technology': [tech_name], 'Country': [tech_country], 'Reason for Drop': ['error in fitting'], 'Number of Years': [len(tech_years)]})
            drop_df = pd.concat([drop_df, new_data], ignore_index = True)
            #save_scatter_plot(tech_years, tech_values, tech_name, tech_country)

        ##### GOMPERTZ FUNCTION #####
        # same parameter starts as logistic function.
        try:
            gomp_params, _ = curve_fit(gompertz_func, tech_years, tech_values, param_start, maxfev=5000)
            g_l_fit, g_k_fit, g_thalf_fit = gomp_params
            gomp_steepness = g_k_fit
            gomp_fit = gompertz_func(tech_years, *gomp_params)
            gomp_r2 = r2_score(tech_values, gomp_fit) if gomp_fit is not None else None
            if gomp_r2 < 0.95: 
                count_badr2_gomp += 1 # add counter for bad R2, will drop.

        except Exception as e:
            gomp_steepness = 'err'
            #print(f'Error for Gompertz occurred for index {idx}: {e}')
            gomp_fit = None
            count_no_gomp_fit += 1
        num_points = len(tech_years)
        first_year = tech_years[0]
        new_tech = {'Variable': id,
                    'Technology Name': tech_name,
                    'Country Code': tech_countrycode,
                   'Country Name': tech_country,
                   'Metric': tech_metric,
                   'Exponential Fit': tech_steepness, 
                   'exponential goodness': exp_r2,
                   'Logistic Fit': log_steepness,
                   'Logistic Saturation': l_fit,
                    'Logistic Inflection Year': thalf_fit,
                    'logistic goodness': log_R_sq,
                   'Gompertz Fit': gomp_steepness,
                   'gompertz goodness': gomp_r2,
                   'Delta T': deltaTspeed,
                   'Vintage': first_year, 
                   'Number of Points': num_points }
        tech_fit_list.append(pd.DataFrame(new_tech, index=[0]))
        #save_time_series_plot(tech_years, tech_values, exp_fit, log_fit, gomp_fit, tech_name, tech_country, exp_r2, log_R_sq, gomp_r2)

    else: 
      #  Print(f'No fit possible for {tech_name} in {tech_country} because not enough points')
        count_no_points += 1
        new_data = pd.DataFrame({'Technology': [tech_name], 'Country': [tech_country], 'Reason for Drop': ['not enough points'], 'Number of Years': [len(tech_years)]})
        drop_df = pd.concat([drop_df, new_data], ignore_index = True)



technology_exp_fits = pd.concat(tech_fit_list, ignore_index = True)


print(f'Not enough points to calculate: {count_no_points}')
print(f'Error in fitting (log): {count_no_log_fit}')
print(f'Error in fitting (exp):{count_no_exp_fit}')
print(f'Error in fitting (Gompertz):{count_no_gomp_fit}')

print(f'Will filter out because negative: {count_negative_log}')

print(f'Will filter out because of bad fit (R2 value): {count_badr2}')
print(f'Bad R2 (Exp): {count_badr2_exp}')
print(f'Bad R2 (Gompertz): {count_badr2_gomp}')

print(f'Growth is too fast: {count_above_1}')

drop_df
#technology_exp_fits




Exception occured: Optimal parameters not found: Number of calls to function has reached maxfev = 5000.
Exception occured: Optimal parameters not found: Number of calls to function has reached maxfev = 5000.
Exception occured: Optimal parameters not found: Number of calls to function has reached maxfev = 5000.
Exception occured: Optimal parameters not found: Number of calls to function has reached maxfev = 5000.
Exception occured: Optimal parameters not found: Number of calls to function has reached maxfev = 5000.
Exception occured: Optimal parameters not found: Number of calls to function has reached maxfev = 5000.
Exception occured: Optimal parameters not found: Number of calls to function has reached maxfev = 5000.
Exception occured: Optimal parameters not found: Number of calls to function has reached maxfev = 5000.
Exception occured: Optimal parameters not found: Number of calls to function has reached maxfev = 5000.
Exception occured: Optimal parameters not found: Number of calls

,Technology,Country,Reason for Drop,Number of Years
0,Postal Traffic,Norway,error in fitting,146
1,Oil Pipelines,Austria,error in fitting,54
2,Oil Pipelines,New Zealand,error in fitting,39
3,Oil Pipelines,Italy,error in fitting,23
4,Oil Pipelines,Japan,error in fitting,34
...,...,...,...,...
197,Raw Steel Production,Ghana,error in fitting,10
198,Crop Harvester,Uzbekistan,error in fitting,10
199,BCG Vaccine,Austria,error in fitting,10
200,MCV2 Vaccine,Djibouti,error in fitting,10


In [21]:
drop_df.to_csv("dropped_reasons_Updated.csv")

In [22]:
save_filename = "technology_growth_Updated.csv"
save_path = os.path.join(save_filename)                
technology_exp_fits.to_csv(save_path)

## Create summary table with key values per technology-country pair

In [23]:
country_tech_summary = pd.DataFrame(columns = [ 'Technology Name', 
                                                  'Country Name', 
                                                  'num values', 
                                                  'num years',
                                                  'First Year of Data',
                                                  'First Value',
                                                  'Last Year of Data',
                                                  'Last Value'])
tech_char_list = []

for index in tech_timeseries_nosum.index:

    years, values = get_time_series(tech_timeseries_nosum, index)
    num_values = len(values)
    num_years = len(years)
    first_year = find_first_year(tech_timeseries_nosum, index)
    if first_year == None:
        first_year_val = None
    else:
        first_year_val = tech_timeseries_nosum[first_year][index]
    last_year = find_last_year(tech_timeseries_nosum, index)
    if last_year == None:
        last_year_val = None
    else:
        last_year_val = tech_timeseries_nosum[last_year][index]
    technology_name = tech_timeseries_sum['Technology Name'][index]
    region_name = tech_timeseries_sum['Country Code'][index]
    new_tech = {'Technology Name': technology_name, 
               'Country Name': region_name,
                'num values': num_values,
                'num years': num_years,
               'First Year of Data': first_year, 
                'First Value': first_year_val,
               'Last Year of Data': last_year,
               'Last Value': last_year_val}
    tech_char_list.append(pd.DataFrame(new_tech, index=[0]))

country_tech_summary = pd.concat(tech_char_list, ignore_index = True)

country_tech_summary

,Technology Name,Country Name,num values,num years,First Year of Data,First Value,Last Year of Data,Last Value
0,Sugar Output,JAM,254,254,1750,21.00,2010,196.800
1,Sugar Output,BRB,233,233,1750,7.00,2010,32.000
2,Sugar Output,CUB,203,203,1786,13.00,2010,1130.000
3,Beer Production,GBR,202,202,1750,948.00,2004,73622.000
4,Sugar Output,MUS,198,198,1813,0.50,2010,436.583
...,...,...,...,...,...,...,...,...
6027,Crop Harvester,UZB,10,10,1992,8000.00,2001,7000.000
6028,Television,BIH,10,10,1991,385612.80,2000,462808.800
6029,BCG Vaccine,AUT,10,10,1980,0.90,1989,0.900
6030,MCV2 Vaccine,DJI,10,10,2012,0.82,2021,0.480


In [24]:
sorted_country_tech_char = country_tech_summary.sort_values('Technology Name')
sorted_country_tech_char

save_filename = "sorted_tech_characteristics_Updated.csv"
save_path = os.path.join("summary_tables", save_filename)                
sorted_country_tech_char.to_csv(save_path)                